### Dependencies

In [ ]:
! pip install --upgrade torch pandas transformers

In [ ]:
! pip install git+https://github.com/adapter-hub/adapter-transformers.git

**RESTART RUNTIME**

### Dataset Prep

In [3]:
! wget "https://github.com/UBC-NLP/aoc_id/raw/master/data/train/MultiTrain.Shuffled.csv"

--2020-09-22 19:00:17--  https://github.com/UBC-NLP/aoc_id/raw/master/data/train/MultiTrain.Shuffled.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UBC-NLP/aoc_id/master/data/train/MultiTrain.Shuffled.csv [following]
--2020-09-22 19:00:17--  https://raw.githubusercontent.com/UBC-NLP/aoc_id/master/data/train/MultiTrain.Shuffled.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14781520 (14M) [text/plain]
Saving to: ‘MultiTrain.Shuffled.csv’

MultiTrain.Shuffled 100%[===================>]  14.10M  72.0MB/s    in 0.2s    

2020-09-22 19:00:17 (72.0 MB/s) - ‘MultiTrain.Shuffled.csv’ saved [1478152

In [1]:
import pandas as pd

df_ubc = pd.read_csv("./MultiTrain.Shuffled.csv", index_col=0)
df_ubc = df_ubc[["text", "label"]]
df_ubc.head()

,text,label
0,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...,MSA
1,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...,MSA
2,لمسه اليد مرتين واضحة جدا والحكم,DIAL_LEV
3,بخصوص الهاتريك عمرها ما راح تصير,DIAL_LEV
4,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،,DIAL_GLF


In [ ]:
! git clone https://github.com/ryancotterell/arabic_dialect_annotation
! gunzip arabic_dialect_annotation/annotated_data.tar.gz
! tar -xvf arabic_dialect_annotation/annotated_data.tar

In [3]:
df_egy = pd.read_csv("annotated_data/egyptian", delimiter="\t").dropna()
df_egy["label"] = "DIAL_EGY"
df_egy = df_egy[["text", "label"]]
df_egy.head(3)

,text,label
1,والله الاخلاق في ها الزمن ضاعت وصارت النساء مث...,DIAL_EGY
2,بصراحة الوحدات الان بدفع ثمن تفريطو بأكثر من ل...,DIAL_EGY
3,الكل ضد المارد الاخضر الجميع فرحان بخساره الوح...,DIAL_EGY


In [4]:
df_lev = pd.read_csv("annotated_data/levantine", delimiter="\t").dropna()
df_lev["label"] = "DIAL_LEV"
df_lev = df_lev[["text", "label"]]

# df_msa = pd.read_csv("annotated_data/msa", delimiter="\t").dropna()
# df_msa["label"] = "MSA"
# df_msa = df_msa[["text", "label"]].sample(20000)

df_mgh = pd.read_csv("annotated_data/maghrebi", delimiter="\t").dropna()
df_mgh["label"] = "MAGHREBI"
df_mgh = df_mgh[["text", "label"]]

df_mgh2 = pd.read_csv("annotated_data/twitter-maghrebi", delimiter="\t").dropna()
df_mgh2["label"] = "MAGHREBI"
df_mgh2 = df_mgh2[["text", "label"]]
df_mgh = df_mgh.append(df_mgh2)

df_glf = pd.read_csv("annotated_data/egyptian", delimiter="\t").dropna()
df_glf["label"] = "DIAL_GLF"
df_glf = df_glf[["text", "label"]]

In [5]:
df = df_ubc.append(df_egy).append(df_lev).append(df_mgh).append(df_glf) #no additional msa
print(len(df))
df.head()

115810


,text,label
0,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...,MSA
1,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...,MSA
2,لمسه اليد مرتين واضحة جدا والحكم,DIAL_LEV
3,بخصوص الهاتريك عمرها ما راح تصير,DIAL_LEV
4,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،,DIAL_GLF


In [6]:
df.label.value_counts()

MSA         50845
DIAL_GLF    25517
DIAL_EGY    18946
DIAL_LEV    13890
MAGHREBI     6612
Name: label, dtype: int64

In [7]:
df.label = pd.Categorical(df.label)
df.label = df.label.cat.codes

### Train/test split combined data
Tokenize / encode

In [8]:
from sklearn.model_selection import train_test_split
train, eval = train_test_split(df)

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')

train_str = []
for line in list(train["text"].astype('str')):
  train_str.append( tokenizer.encode(line, truncation=True, max_length=128, pad_to_max_length=True) )

eval_str = []
for line in list(eval["text"].astype('str')):
  eval_str.append( tokenizer.encode(line, truncation=True, max_length=128, pad_to_max_length=True) )

### Prepare AdapterHub

In [10]:
from transformers import AutoModelWithHeads
model = AutoModelWithHeads.from_pretrained('aubmindlab/bert-base-arabert')

from transformers import AdapterType
model.add_adapter("dialect-arabic", AdapterType.text_task)
model.train_adapter(["dialect-arabic"])
model.add_classification_head("dialect-arabic", num_labels=5)
model.set_active_adapters([["dialect-arabic"]])

In [11]:
import torch
from torch.utils.data import Dataset

class Result():
    def __init__(self, x, y):
        self.input_ids = x
        self.label = y

class DatasetHelper(Dataset):
    def __init__(self, encodings, labels):
        self.input_ids = encodings
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return Result(self.input_ids[idx], self.labels[idx])

### Run training

In [12]:
from transformers import Trainer, TrainingArguments, EvalPrediction, glue_compute_metrics

# def compute_metrics(p: EvalPrediction):
#     preds = np.argmax(p.predictions, axis=1)
#     return glue_compute_metrics(data_args.task_name, preds, p.label_ids)

training_args = TrainingArguments(
    logging_steps=1000, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/dialect-arabic",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.0001,
    num_train_epochs=3,
)

In [13]:
train_dt = DatasetHelper(train_str, list(train["label"]))
eval_dt = DatasetHelper(eval_str, list(eval["label"]))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dt,
    eval_dataset=eval_dt,
    #compute_metrics=compute_metrics,
    #tokenizer=tokenizer
)

In [14]:
trainer.train()
trainer.evaluate()

{"loss": 1.0211266680657864, "learning_rate": 8.772252915899325e-05, "epoch": 0.3683241252302026, "step": 1000}



{"eval_loss": 0.8534925939231519, "epoch": 0.3683241252302026, "step": 1000}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 0.8500486992001534, "learning_rate": 7.54450583179865e-05, "epoch": 0.7366482504604052, "step": 2000}



{"eval_loss": 0.8028402879143393, "epoch": 0.7366482504604052, "step": 2000}



...


{"eval_loss": 0.7234259271463811, "epoch": 2.2099447513812156, "step": 6000}
{"loss": 0.7075126309394837, "learning_rate": 1.4057704112952733e-05, "epoch": 2.578268876611418, "step": 7000}



{"eval_loss": 0.723701252513637, "epoch": 2.578268876611418, "step": 7000}
{"loss": 0.7032528442144393, "learning_rate": 1.7802332719459791e-06, "epoch": 2.9465930018416207, "step": 8000}



{"eval_loss": 0.7146557684634168, "epoch": 2.9465930018416207, "step": 8000}





{"eval_loss": 0.7137831356088583, "epoch": 3.0, "step": 8145}


{'epoch': 3.0, 'eval_loss': 0.7137831356088583}

### Export Adapter

In [15]:
model.save_adapter("./dialect-ar", "dialect-arabic")